In [41]:
import pandas as pd

from dateutil import parser
from datetime import datetime
from dateutil.relativedelta import relativedelta

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

In [42]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [43]:
df = pd.read_sql('SELECT * FROM symbols', con=database_connection)

In [44]:
del df['index']
df

SYMBOL                           NAME OF COMPANY  SERIES  \
0      20MICRONS                        20 Microns Limited      EQ   
1     21STCENMGM  21st Century Management Services Limited      EQ   
2     3IINFOTECH                       3i Infotech Limited      EQ   
3        3MINDIA                          3M India Limited      EQ   
4         3PLAND                  3P Land Holdings Limited      EQ   
...          ...                                       ...     ...   
1648   ZODJRDMKJ                   Zodiac JRD- MKJ Limited      EQ   
1649        ZOTA                  Zota Health Care LImited      EQ   
1650       ZUARI              Zuari Agro Chemicals Limited      EQ   
1651   ZUARIGLOB                      Zuari Global Limited      EQ   
1652   ZYDUSWELL                    Zydus Wellness Limited      EQ   

       ISIN NUMBER  
0     INE144J01027  
1     INE253B01015  
2     INE748C01020  
3     INE470A01017  
4     INE105C01023  
...            ...  
1648  INE077B01018  
1649  INE358U01012  
1650  INE840M01016  
1651  INE217A01012  
1652  INE768C01010  

[1653 rows x 4 columns]

In [45]:
df2 = pd.read_sql('SELECT distinct company_name FROM mainschema.marketsmojorecos',con=database_connection)

In [46]:
df2

company_name
0    Colgate Palmol. (I)
1        Larsen & Toubro
2         Britannia Inds
3    Sahyadri Industries
4         Tata Chemicals
..                   ...
782    Motilal Oswal Fin
783       Globus Spirits
784      Orient Electric
785        Polycab India
786            Adani Gas

[787 rows x 1 columns]

In [47]:
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

In [48]:
stripped = []
for i in df2['company_name'] :
    i = i.lower()
    i = i.replace(" ","")
    i = i.replace(".","")
    i = i.replace("(i)","")
    i = i.replace("(","")
    i = i.replace(")","")
    i = i.replace("-","")
    stripped.append(i)

In [49]:
stripped2 = []
for i in df['NAME OF COMPANY'] :
    i = i.lower()
    i = i.replace(" ","")
    i = i.replace(".","")
    i = i.replace("(i)","")
    i = i.replace("(","")
    i = i.replace(")","")
    i = i.replace("-","")
    stripped2.append(i)

In [ ]:
y=0
z=0
ilist=[]
matchlist=[]
iNotFoundList=[]
for i in stripped:
    x=0
    for j in stripped2:
        Partial_Ratio = fuzz.partial_ratio(i,j)
        if Partial_Ratio > 85:
            x=x+1
            match = j
            matchratio = Partial_Ratio
    if x==1:
        #print(i,"||",match,"||",matchratio)
        ilist.append(i)
        matchlist.append(match)
        y=y+1
    else:
        iNotFoundList.append(i)
        z=z+1

In [ ]:
dfinal = pd.DataFrame({'recoName':ilist, 'formalName':matchlist})

In [19]:
dNotFound = pd.DataFrame({'recoName':iNotFoundList})

In [22]:
tot = y+z
tot

787

In [24]:
df3=df

In [26]:
stripped3 = []
for i in df3['NAME OF COMPANY'] :
    i = i.lower()
    i = i.replace(" ","")
    i = i.replace(".","")
    i = i.replace("(i)","")
    i = i.replace("(","")
    i = i.replace(")","")
    i = i.replace("-","")
    stripped3.append(i)
df3['formalName'] = stripped3

In [28]:
del df3['NAME OF COMPANY']
df3

SYMBOL  SERIES   ISIN NUMBER                            formalName
0      20MICRONS      EQ  INE144J01027                      20micronslimited
1     21STCENMGM      EQ  INE253B01015  21stcenturymanagementserviceslimited
2     3IINFOTECH      EQ  INE748C01020                     3iinfotechlimited
3        3MINDIA      EQ  INE470A01017                        3mindialimited
4         3PLAND      EQ  INE105C01023                 3plandholdingslimited
...          ...     ...           ...                                   ...
1648   ZODJRDMKJ      EQ  INE077B01018                   zodiacjrdmkjlimited
1649        ZOTA      EQ  INE358U01012                 zotahealthcarelimited
1650       ZUARI      EQ  INE840M01016             zuariagrochemicalslimited
1651   ZUARIGLOB      EQ  INE217A01012                    zuarigloballimited
1652   ZYDUSWELL      EQ  INE768C01010                  zyduswellnesslimited

[1653 rows x 4 columns]

In [36]:
dfinal = dfinal.merge(df3,on = 'formalName', how = 'inner')
dfinal

recoName                            formalName      SYMBOL  \
0        colgatepalmol          colgatepalmoliveindialimited      COLPAL   
1        britanniainds            britanniaindustrieslimited   BRITANNIA   
2        tatachemicals                  tatachemicalslimited    TATACHEM   
3         marutisuzuki              marutisuzukiindialimited      MARUTI   
4    manappuramfinance              manappuramfinancelimited  MANAPPURAM   
..                 ...                                   ...         ...   
560    motilaloswalfin  motilaloswalfinancialserviceslimited  MOTILALOFS   
561      globusspirits                  globusspiritslimited   GLOBUSSPR   
562     orientelectric                 orientelectriclimited  ORIENTELEC   
563       polycabindia                   polycabindialimited     POLYCAB   
564           adanigas                       adanigaslimited    ADANIGAS   

     SERIES   ISIN NUMBER  
0        EQ  INE259A01022  
1        EQ  INE216A01030  
2        EQ  INE092A01019  
3        EQ  INE585B01010  
4        EQ  INE522D01027  
..      ...           ...  
560      EQ  INE338I01027  
561      EQ  INE615I01010  
562      EQ  INE142Z01019  
563      EQ  INE455K01017  
564      EQ  INE399L01023  

[565 rows x 5 columns]

In [39]:
dfinal.to_sql(con=database_connection, name='symbolsmapped', if_exists='append')

In [40]:
dNotFound.to_sql(con=database_connection, name='symbolsnotmapped', if_exists='append')